In [1]:
!pip install datasets

In [2]:
!pip install langchain

In [3]:
import os
import json
import time
import pandas as pd
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
# from tqdm import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_experimental.text_splitters import SentenceTransformersTokenTextSplitter
from langchain_text_splitters import SentenceTransformersTokenTextSplitter
from datasets import load_dataset
import pandas as pd
import chromadb
from chromadb.config import Settings
import os
import json
from datasets import load_dataset


In [4]:
# from datasets import load_dataset
# import pandas as pd

# train_stream = load_dataset(
#     "TechQueen24/DoxplainQA",
#     split="train",
#     streaming=True
# )

# df = pd.DataFrame(list(train_stream))
# print(df.shape)
# df.head()


In [5]:
hotpotqa = load_dataset("hotpotqa/hotpot_qa", "fullwiki")

README.md: 0.00B [00:00, ?B/s]

fullwiki/train-00000-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

fullwiki/train-00001-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

fullwiki/validation-00000-of-00001.parqu(…):   0%|          | 0.00/28.0M [00:00<?, ?B/s]

fullwiki/test-00000-of-00001.parquet:   0%|          | 0.00/27.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [6]:
hotpotqa = hotpotqa["train"].to_pandas()
len(hotpotqa)

90447

In [7]:
hotpotqa.iloc[0].context

{'title': array(['Radio City (Indian radio station)',
        'History of Albanian football', 'Echosmith',
        "Women's colleges in the Southern United States",
        'First Arthur County Courthouse and Jail', "Arthur's Magazine",
        '2014–15 Ukrainian Hockey Championship', 'First for Women',
        'Freeway Complex Fire', 'William Rast'], dtype=object),
 'sentences': array([array(["Radio City is India's first private FM radio station and was started on 3 July 2001.",
               ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
               ' It plays Hindi, English and regional songs.',
               ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.',
               ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that

In [8]:
hotpotqa.to_csv("/data/hotpotqa.csv", index=False)

In [16]:
# Configuration
CHROMA_DB_PATH = "/data/chromadb"  # Local path that Docker will mount
HOTPOT_DATA_PATH = "/data/hotpotqa.csv"
COLLECTION_NAME = "hotpotqa"

import os
import ast
import re
import numpy as np
import pandas as pd
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

CHROMA_DB_PATH = "/data/chromadb"
HOTPOT_DATA_PATH = "/data/hotpotqa.csv"
COLLECTION_NAME = "hotpotqa"

CHAR_CHUNK_SIZE = 2000
CHAR_CHUNK_OVERLAP = 200
TOKEN_CHUNK_SIZE = 256
TOKEN_CHUNK_OVERLAP = 32


# -----------------------------
# Helpers
# -----------------------------
def _to_py(obj):
    """Convert numpy arrays / pandas objects into pure Python lists/strings recursively."""
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (list, tuple)):
        return [_to_py(x) for x in obj]
    if isinstance(obj, dict):
        return {k: _to_py(v) for k, v in obj.items()}
    return obj


def _safe_eval_numpy_repr(s: str):
    """
    Safely evaluate a string that contains numpy 'array(...)' repr.
    Works for your exact data format (array([...], dtype=object)).
    """
    # Quick gate
    if "array(" not in s:
        return None

    # Restricted eval environment:
    # - no builtins
    # - only allow numpy.array, numpy.nan, and Python 'object' for dtype=object
    safe_globals = {
        "__builtins__": {},
        "array": np.array,
        "nan": np.nan,
        "object": object,
    }

    # Some exports may have weird unicode spaces; normalize lightly
    s2 = s.strip()

    try:
        return eval(s2, safe_globals, {})
    except Exception:
        return None


def process_context(context_obj):
    """
    Convert HotpotQA 'context' to clean text:
    - If it's a string with numpy array repr -> eval it safely -> tolist()
    - If it's already dict/np arrays -> tolist()
    - Join as:  Title: sentence sentence ...
    """
    # Case 1: already dict-like (from a different loader)
    if isinstance(context_obj, dict):
        ctx = _to_py(context_obj)
        return _context_dict_to_text(ctx)

    # Case 2: string
    if isinstance(context_obj, str):
        s = context_obj.strip()

        # Try literal_eval first (works if it’s a real dict/lists string)
        try:
            obj = ast.literal_eval(s)
            obj = _to_py(obj)
            if isinstance(obj, dict):
                return _context_dict_to_text(obj)
            return str(obj)
        except Exception:
            pass

        # Try numpy-repr eval (this is YOUR current format)
        obj = _safe_eval_numpy_repr(s)
        if isinstance(obj, dict):
            obj = _to_py(obj)
            return _context_dict_to_text(obj)

        # Fallback: return raw string (but at least we won't crash)
        return s

    # Fallback for unexpected types
    return str(context_obj)


def _context_dict_to_text(ctx: dict) -> str:
    titles = ctx.get("title", []) or []
    sentences = ctx.get("sentences", []) or []

    # Ensure list-of-lists
    titles = list(titles) if not isinstance(titles, list) else titles
    sentences = list(sentences) if not isinstance(sentences, list) else sentences

    parts = []
    for title, sents in zip(titles, sentences):
        if isinstance(sents, (list, tuple)):
            paragraph = " ".join(str(x).strip() for x in sents if str(x).strip())
        else:
            paragraph = str(sents).strip()

        t = str(title).strip()
        if t and paragraph:
            parts.append(f"{t}: {paragraph}")
        elif paragraph:
            parts.append(paragraph)

    # This is now CLEAN natural language
    return "\n\n".join(parts).strip()


def chunk_context(context_text: str):
    if not context_text or not context_text.strip():
        return []

    char_chunks = character_splitter.split_text(context_text)
    token_chunks = []
    for c in char_chunks:
        token_chunks.extend(token_splitter.split_text(c))

    return [t.strip() for t in token_chunks if t and t.strip()]


def load_from_dataframe_chunked(df):
    print(f"Processing DataFrame with {len(df)} rows...")

    documents, metadatas, ids = [], [], []

    for row_idx, row in df.iterrows():
        qid = str(row.get("id", row.get("_id", row_idx)))

        context_text = process_context(row["context"])
        chunks = chunk_context(context_text)
        if not chunks:
            continue

        base_meta = {
            "question_id": qid,
            "question": str(row.get("question", "")),
            "answer": str(row.get("answer", "")),
            "type": str(row.get("type", "")),
            "level": str(row.get("level", "")),
        }

        for ci, chunk in enumerate(chunks):
            chunk_id = f"{qid}::c{ci}"
            documents.append(chunk)
            metadatas.append({**base_meta, "chunk_index": ci})
            ids.append(chunk_id)

        if (row_idx + 1) % 100 == 0:
            print(f"  Processed {row_idx + 1}/{len(df)} rows; chunks so far: {len(documents)}")

    return documents, metadatas, ids


def create_chromadb(documents, metadatas, ids, db_path):
    os.makedirs(db_path, exist_ok=True)

    chroma_client = chromadb.PersistentClient(
        path=db_path,
        settings=Settings(anonymized_telemetry=False, allow_reset=True),
    )

    embedding_fn = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

    # Delete if exists (don’t crash on first run)
    try:
        chroma_client.delete_collection(COLLECTION_NAME)
    except Exception:
        pass

    collection = chroma_client.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_fn,
    )

    print(f"Adding {len(documents)} chunks to Chroma...")
    batch_size = 1000
    for i in range(0, len(documents), batch_size):
        j = min(i + batch_size, len(documents))
        collection.add(documents=documents[i:j], metadatas=metadatas[i:j], ids=ids[i:j])
        print(f"  Added {j}/{len(documents)}")

    print("Done. Collection count:", collection.count())
    return collection


# Splitters
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=CHAR_CHUNK_SIZE,
    chunk_overlap=CHAR_CHUNK_OVERLAP,
)

token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=TOKEN_CHUNK_OVERLAP,
    tokens_per_chunk=TOKEN_CHUNK_SIZE,
)


def main():
    df = pd.read_csv(HOTPOT_DATA_PATH)
    df = df[:10000]
    documents, metadatas, ids = load_from_dataframe_chunked(df)
    create_chromadb(documents, metadatas, ids, CHROMA_DB_PATH)


if __name__ == "__main__":
    main()


Processing DataFrame with 10000 rows...
  Processed 100/10000 rows; chunks so far: 762
  Processed 200/10000 rows; chunks so far: 1475
  Processed 300/10000 rows; chunks so far: 2177
  Processed 400/10000 rows; chunks so far: 2907
  Processed 500/10000 rows; chunks so far: 3598
  Processed 600/10000 rows; chunks so far: 4323
  Processed 700/10000 rows; chunks so far: 5038
  Processed 800/10000 rows; chunks so far: 5726
  Processed 900/10000 rows; chunks so far: 6477
  Processed 1000/10000 rows; chunks so far: 7223
  Processed 1100/10000 rows; chunks so far: 7954
  Processed 1200/10000 rows; chunks so far: 8652
  Processed 1300/10000 rows; chunks so far: 9378
  Processed 1400/10000 rows; chunks so far: 10108
  Processed 1500/10000 rows; chunks so far: 10806
  Processed 1600/10000 rows; chunks so far: 11528
  Processed 1700/10000 rows; chunks so far: 12242
  Processed 1800/10000 rows; chunks so far: 12996
  Processed 1900/10000 rows; chunks so far: 13709
  Processed 2000/10000 rows; chun

In [17]:
from datasets import load_dataset

ds = load_dataset("yzhuang/narrative_qa")

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [18]:
narative_qa = ds["train"].to_pandas()
len(narative_qa)

32747

In [19]:
narative_qa.to_csv("/data/narative_qa.csv", index=False)

In [20]:
narative_qa.head()

,id,context,question,answer
0,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",Who is Miss Delmer?,[the elderly spinster aunt of the Earl de Vers...
1,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",Who does Arabella Mason wed?,"[Ben Keene, Delmar's valet, Ben Keene]"
2,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",How does Percival Keene get his name?,"[Percival is Captain Delmar's first name, and ..."
3,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",Who is the bully that steals Percival's lunch?,"[his teacher, Mr. O'Gallagher, The schoolmaster]"
4,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",How does Percival get even with O'Gallagher af...,[He sets them on fire with the teacher sitting...


In [21]:
narative_qa.iloc[0].answer

array(['the elderly spinster aunt of the Earl de Verseley and Captain Delmar',
       "She's Captail Delmar's aunt."], dtype=object)

In [ ]:
#!/usr/bin/env python3
import os
import ast
import numpy as np
import pandas as pd
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

# -----------------------------
# Configuration
# -----------------------------
CHROMA_DB_PATH = os.getenv("CHROMA_DB_PATH", "/data/chromadb")
NARRATIVE_CSV_PATH = os.getenv("NARRATIVE_CSV_PATH", "/data/narative_qa.csv")
COLLECTION_NAME = os.getenv("COLLECTION_NAME", "narrativeqa")

CHAR_CHUNK_SIZE = int(os.getenv("CHAR_CHUNK_SIZE", "2000"))
CHAR_CHUNK_OVERLAP = int(os.getenv("CHAR_CHUNK_OVERLAP", "200"))
TOKEN_CHUNK_SIZE = int(os.getenv("TOKEN_CHUNK_SIZE", "256"))
TOKEN_CHUNK_OVERLAP = int(os.getenv("TOKEN_CHUNK_OVERLAP", "32"))

MAX_ROWS = os.getenv("MAX_ROWS", "10000")  # "None" or integer
MAX_ROWS = None if str(MAX_ROWS).lower() == "none" else int(MAX_ROWS)

EMBED_MODEL = os.getenv("EMBED_MODEL", "all-MiniLM-L6-v2")

# -----------------------------
# Splitters
# -----------------------------
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=CHAR_CHUNK_SIZE,
    chunk_overlap=CHAR_CHUNK_OVERLAP,
)

token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=TOKEN_CHUNK_OVERLAP,
    tokens_per_chunk=TOKEN_CHUNK_SIZE,
)

# -----------------------------
# Helpers
# -----------------------------
def _safe_eval_numpy_array_repr(s: str):
    if not isinstance(s, str):
        return None
    if "array(" not in s:
        return None

    safe_globals = {
        "__builtins__": {},
        "array": np.array,
        "nan": np.nan,
        "object": object,
    }
    try:
        return eval(s.strip(), safe_globals, {})
    except Exception:
        return None

def parse_answer_first(answer_obj) -> str:
    answers = []

    if isinstance(answer_obj, np.ndarray):
        answers = answer_obj.tolist()
    elif isinstance(answer_obj, (list, tuple)):
        answers = list(answer_obj)
    elif isinstance(answer_obj, str):
        s = answer_obj.strip()
        try:
            obj = ast.literal_eval(s)
            if isinstance(obj, (list, tuple)):
                answers = list(obj)
            else:
                answers = [obj]
        except Exception:
            arr = _safe_eval_numpy_array_repr(s)
            if isinstance(arr, np.ndarray):
                answers = arr.tolist()
            else:
                answers = [s] if s else []
    else:
        answers = [answer_obj]

    answers = [str(a).strip() for a in answers if a is not None and str(a).strip()]
    return answers[0] if answers else ""

def clean_context(context: str) -> str:
    return "" if context is None else str(context).strip()

def chunk_text(text: str):
    if not text or not text.strip():
        return []
    char_chunks = character_splitter.split_text(text)
    token_chunks = []
    for c in char_chunks:
        token_chunks.extend(token_splitter.split_text(c))
    return [t.strip() for t in token_chunks if t and t.strip()]

# -----------------------------
# Build docs/metas/ids
# -----------------------------
def build_records(df: pd.DataFrame):
    documents, metadatas, ids = [], [], []

    for row_idx, row in df.iterrows():
        story_id = str(row.get("id", row.get("_id", "unknown"))).strip()

        # ✅ UNIQUE per-question id (story id + row index)
        question_id = f"{story_id}::q{row_idx}"

        question = str(row.get("question", "")).strip()
        context_text = clean_context(row.get("context", ""))

        chunks = chunk_text(context_text)
        if not chunks:
            continue

        answer = parse_answer_first(row.get("answer", ""))

        base_meta = {
            "question_id": question_id,   # ✅ unique id per question (important!)
            "story_id": story_id,         # keep original doc/story id
            "question": question,
            "answer": answer,
            "dataset": "narrativeqa",
        }

        for ci, chunk in enumerate(chunks):
            # ✅ chunk ids now unique per question (no collisions)
            chunk_id = f"{question_id}::c{ci}"
            documents.append(chunk)
            metadatas.append({**base_meta, "chunk_index": int(ci)})
            ids.append(chunk_id)

        if (row_idx + 1) % 100 == 0:
            print(f"  Processed {row_idx + 1}/{len(df)} rows; chunks so far: {len(documents)}")

    return documents, metadatas, ids

# -----------------------------
# Write Chroma collection
# -----------------------------
def write_chroma(documents, metadatas, ids):
    os.makedirs(CHROMA_DB_PATH, exist_ok=True)

    client = chromadb.PersistentClient(
        path=CHROMA_DB_PATH,
        settings=Settings(anonymized_telemetry=False, allow_reset=True),
    )

    embedding_fn = SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)

    try:
        client.delete_collection(COLLECTION_NAME)
    except Exception:
        pass

    col = client.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_fn,
    )

    print(f"Adding {len(documents)} chunks to Chroma collection '{COLLECTION_NAME}' ...")
    batch_size = 1000
    for i in range(0, len(documents), batch_size):
        j = min(i + batch_size, len(documents))
        col.add(documents=documents[i:j], metadatas=metadatas[i:j], ids=ids[i:j])
        print(f"  Added {j}/{len(documents)}")

    print("Done. Collection count:", col.count())
    return col

def main():
    df = pd.read_csv(NARRATIVE_CSV_PATH)

    df = df[:10000]
    if MAX_ROWS is not None:
        df = df.iloc[:MAX_ROWS].copy()

    print(f"Loaded NarrativeQA rows: {len(df)} from {NARRATIVE_CSV_PATH}")
    docs, metas, ids = build_records(df)
    write_chroma(docs, metas, ids)

if __name__ == "__main__":
    main()


Loaded NarrativeQA rows: 10000 from /data/narative_qa.csv
  Processed 100/10000 rows; chunks so far: 53801
  Processed 200/10000 rows; chunks so far: 83063
